In [1]:
import pandas as pd
import re

In [2]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [3]:
df = pd.read_csv('../raw_data/database_entities.csv')
df

,Unnamed: 0,entity,score,index,word,start,end,line_id
0,0,B-PER,0.915520,4,V,11,12,0
1,1,B-LOC,0.999274,17,American,51,59,0
2,2,I-LOC,0.998381,18,Samoa,60,65,0
3,3,B-MISC,0.999530,29,Vietnamese,104,114,0
4,4,B-ORG,0.973456,46,Tour,196,200,0
...,...,...,...,...,...,...,...,...
15967,15967,I-LOC,0.996284,223,##z,970,971,1357
15968,15968,B-LOC,0.999348,284,A,1227,1228,1357
15969,15969,I-LOC,0.874068,285,##gade,1228,1232,1357
15970,15970,I-LOC,0.987688,286,##z,1232,1233,1357


In [4]:
df = df.drop(columns = ['Unnamed: 0', 'index', 'start', 'end'])
df

,entity,score,word,line_id
0,B-PER,0.915520,V,0
1,B-LOC,0.999274,American,0
2,I-LOC,0.998381,Samoa,0
3,B-MISC,0.999530,Vietnamese,0
4,B-ORG,0.973456,Tour,0
...,...,...,...,...
15967,I-LOC,0.996284,##z,1357
15968,B-LOC,0.999348,A,1357
15969,I-LOC,0.874068,##gade,1357
15970,I-LOC,0.987688,##z,1357


In [5]:
df = df[(df["entity"]=="B-LOC")|(df["entity"]=="I-LOC")][df["score"]>0.98]

/var/folders/rv/1874f2853zqf1lx4mr_wkbsc0000gn/T/ipykernel_19789/4092738895.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[(df["entity"]=="B-LOC")|(df["entity"]=="I-LOC")][df["score"]>0.98]


In [6]:
df.rename(columns={"line_id": "victim_id"})

,entity,score,word,victim_id
1,B-LOC,0.999274,American,0
2,I-LOC,0.998381,Samoa,0
7,B-LOC,0.999507,US,0
9,B-LOC,0.999251,American,0
10,I-LOC,0.998292,Samoa,0
...,...,...,...,...
15965,B-LOC,0.999359,A,1357
15967,I-LOC,0.996284,##z,1357
15968,B-LOC,0.999348,A,1357
15970,I-LOC,0.987688,##z,1357


In [7]:
r=list(df['word'].unique())

In [8]:
s=list()
for i in r:
    if len(re.findall("#.*",i))==0:
        s.append(i)
    else:
        pass
s    

['American',
 'Samoa',
 'US',
 'Col',
 'Sri',
 'Lanka',
 'Lebanon',
 'Beirut',
 'Allah',
 'Ka',
 'N',
 'G',
 'K',
 'Cameroon',
 'United',
 'States',
 'America',
 'Minnesota',
 'of',
 'Haiti',
 'Port',
 '-',
 'au',
 'Prince',
 'Mexico',
 'Arizona',
 'Florida',
 'No',
 'Ma',
 'Bar',
 'Ram',
 'Son',
 'Uttar',
 'Pradesh',
 'India',
 'San',
 'Niger',
 'Mali',
 'Egypt',
 'Paris',
 'France',
 'Myrtle',
 'Beach',
 'SC',
 'Miami',
 'FL',
 'Houston',
 'TX',
 'Diego',
 'CA',
 'New',
 'Orleans',
 'LA',
 'Scott',
 'A',
 'Chicago',
 'IL',
 'Columbus',
 'OH',
 'Denver',
 'CO',
 'Silver',
 'Spring',
 'England',
 'Koch',
 'Nigeria',
 'Europe',
 'South',
 'Africa',
 'Johannesburg',
 'B',
 'Be',
 'Pieter',
 'Hill',
 'Los',
 'Angeles',
 'U',
 'S',
 '.',
 'Indonesia',
 'York',
 'City',
 'NY',
 'Mom',
 'Southern',
 'Boston',
 'Philippines',
 'California',
 'North',
 'Korea',
 'China',
 'Tu',
 'River',
 'USA',
 'Beijing',
 'Guangdong',
 'Province',
 'To',
 'Ko',
 'Ghana',
 'Northern',
 'Sudan',
 'Ukraine',
 

In [9]:
canc=list()
for i in s:
    if len(i)==1:
        canc.append(i)
    try:
        int(i)
        canc.append(i)
    except:
        pass
    
        

In [11]:
def transform(item):
    if item in canc:
        item=item.replace(item,"np.nan")
    return item
df["word"]=df["word"].apply(transform)      

In [12]:
import numpy as np
df["word"]=df["word"].replace("#.*",np.nan,regex=True)

In [13]:
df=df.dropna(subset=["word"])

In [15]:
df=df[df["word"]!="np.nan"]

In [16]:
df.drop(columns="score",inplace=True)

/Users/sofiagiordano/.pyenv/versions/3.8.12/envs/ht_project/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [17]:
df.drop(columns="entity",inplace=True)

In [21]:
df.rename(columns={"word":"location", "line_id":"victim_index"},inplace=True)

/Users/sofiagiordano/.pyenv/versions/3.8.12/envs/ht_project/lib/python3.8/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [22]:
df

,location,victim_index
1,American,0
2,Samoa,0
7,US,0
9,American,0
10,Samoa,0
...,...,...
15958,Libya,1356
15959,Benin,1357
15960,City,1357
15964,Niger,1357


In [23]:
df.to_csv('victims_locations.csv')